In [14]:
# 1
import numpy as np

# --- 1. 已知参数 ---
A = 1.0   # m
B = 0.6   # m
C = 2.5   # m
xp = -1.2 # m (P点在O4左侧，所以为负)
vp = 3.0 # m/s (滑块向左移动，所以为负)
ap =1.0 # m/s^2 (滑块向左加速，所以为负)

# --- 2. 位置分析 (Position Analysis) ---
# B^2 - 2*B*xp*cos(theta_B) + xp^2 = A^2
# => cos(theta_B) = (xp^2 + B^2 - A^2) / (2 * B * xp)
cos_theta_B = (xp**2 + B**2 - A**2) / (2 * B * xp)
# 根据图示，theta_B 应该在第二象限 (90-180度)，arccos的结果正好在此范围内
theta_B = np.arccos(cos_theta_B)
sin_theta_B = np.sin(theta_B)

# 求解 theta_A
# A*cos(theta_A) = B*cos(theta_B) - xp
# A*sin(theta_A) = B*sin(theta_B)
y_comp_A = B * sin_theta_B
x_comp_A = B * cos_theta_B - xp
theta_A = np.arctan2(y_comp_A, x_comp_A)
cos_theta_A = np.cos(theta_A)
sin_theta_A = np.sin(theta_A)

# --- 3. 速度分析 (Velocity Analysis) ---
# 求解 [omega_A, omega_B] 的线性方程组
# -A*sin(theta_A)*wA + B*sin(theta_B)*wB = -vp
#  A*cos(theta_A)*wA - B*cos(theta_B)*wB = 0
M = np.array([
    [-A * sin_theta_A, B * sin_theta_B],
    [A * cos_theta_A, -B * cos_theta_B]
])
b_vel = np.array([-vp, 0])

# 求解角速度
omega_A, omega_B = np.linalg.solve(M, b_vel)

# --- 4. 加速度分析 (Acceleration Analysis) ---
# 求解 [alpha_A, alpha_B] 的线性方程组
# 系数矩阵 M 与速度分析相同
# RHS (Right Hand Side) of the equation:
# b_accel_1 = -ap + wA^2*A*cos(theta_A) - wB^2*B*cos(theta_B)
# b_accel_2 = wA^2*A*sin(theta_A) - wB^2*B*sin(theta_B)
b_accel = np.array([
    -ap + omega_A**2 * A * cos_theta_A - omega_B**2 * B * cos_theta_B,
    omega_A**2 * A * sin_theta_A - omega_B**2 * B * sin_theta_B
])

# 求解角加速度
alpha_A, alpha_B = np.linalg.solve(M, b_accel)

# --- 5. 计算水球的运动学参数 ---
# 使用复数表示法进行计算
# r_S = (B+C) * exp(j*theta_B)

r_S_vec = (C) * np.exp(1j * theta_B)

# v_S = j*omega_B * r_S
v_S_vec = 1j * omega_B * r_S_vec

# a_S = (j*alpha_B - omega_B^2) * r_S
a_S_vec = (1j * alpha_B - omega_B**2) * r_S_vec

# --- 6. 打印结果 ---
print("--- PVA 分析结果 ---")
print("\n--- 连杆角度 ---")
print(f"连杆 A 的角度 (theta_A): {np.rad2deg(theta_A):.2f} 度")
print(f"连杆 B 的角度 (theta_B): {np.rad2deg(theta_B):.2f} 度")

print("\n--- 连杆角速度 ---")
print(f"连杆 A 的角速度 (omega_A): {omega_A:.4f} rad/s")
print(f"连杆 B 的角速度 (omega_B): {omega_B:.4f} rad/s")

print("\n--- 连杆角加速度 ---")
print(f"连杆 A 的角加速度 (alpha_A): {alpha_A:.4f} rad/s^2")
print(f"连杆 B 的角加速度 (alpha_B): {alpha_B:.4f} rad/s^2")

print("\n--- 水球的运动状态 ---")
print(f"水球位置 (x, y): ({r_S_vec.real:.3f}, {r_S_vec.imag:.3f}) m")
print(f"水球线速度 (vx, vy): ({v_S_vec.real:.3f}, {v_S_vec.imag:.3f}) m/s")
print(f"水球线速度大小: {np.abs(v_S_vec):.3f} m/s")
print(f"水球线加速度 (ax, ay): ({a_S_vec.real:.3f}, {a_S_vec.imag:.3f}) m/s^2")
print(f"水球线加速度大小: {np.abs(a_S_vec):.3f} m/s^2")


--- PVA 分析结果 ---

--- 连杆角度 ---
连杆 A 的角度 (theta_A): 29.93 度
连杆 B 的角度 (theta_B): 123.75 度

--- 连杆角速度 ---
连杆 A 的角速度 (omega_A): 1.6704 rad/s
连杆 B 的角速度 (omega_B): -4.3430 rad/s

--- 连杆角加速度 ---
连杆 A 的角加速度 (alpha_A): -10.9718 rad/s^2
连杆 B 的角加速度 (alpha_B): 4.4732 rad/s^2

--- 水球的运动状态 ---
水球位置 (x, y): (-1.389, 2.079) m
水球线速度 (vx, vy): (9.028, 6.032) m/s
水球线速度大小: 10.857 m/s
水球线加速度 (ax, ay): (16.898, -45.420) m/s^2
水球线加速度大小: 48.462 m/s^2


In [10]:
from sympy import *
w1,w3=S('w1,w3')
q1=3-A*w1*sin(theta_A)+B*w3*sin(theta_B)
q2=A*w1*cos(theta_A)-B*w3*cos(theta_B)
an=solve([q1,q2],[w1,w3])
w1=an[w1]
w3=an[w3]

a1,a3=S('a1,a3')
q1=1-a1*A*sin(theta_A)+a3*B*sin(theta_B)-A*w1**2*cos(theta_A)+B*w3**2*cos(theta_B)
q2=a1*A*cos(theta_A)-a3*B*cos(theta_B)-A*w1**2*sin(theta_A)+B*w3**2*sin(theta_B)
solve([q1,q2],[a1,a3])



{a1: -10.9718296298497, a3: 4.47324526549106}

In [28]:
import numpy as np
from scipy.optimize import fsolve

def solve_general_kinematics(R1, R2, P_c, V_c, A_c):
    """
    根据给定的驱动点C的完整2D运动状态，求解机构的运动学。

    参数:
    R1 (float): 连杆1的长度 (连接驱动点C)
    R2 (float): 连杆2的长度 (连接地面)
    P_c (tuple): C点的位置矢量 (xc, yc)
    V_c (tuple): C点的速度矢量 (v_cx, v_cy)
    A_c (tuple): C点的加速度矢量 (a_cx, a_cy)

    返回:
    一个包含所有求解结果的字典
    """
    xc, yc = P_c
    v_cx, v_cy = V_c
    a_cx, a_cy = A_c

    # --- 1. 位置分析 (求解 theta_1, theta_2) ---
    # 使用数值求解器求解非线性方程组
    # xc+R1*cos(th1) - R2*cos(th2) = 1.5
    # yc+R1*sin(th1) - R2*sin(th2) = 0c
    def position_equations(vars):
        th1, th2 = vars
        eq1 = R1 * np.cos(th1) - R2 * np.cos(th2) + xc-1.5
        eq2 = R1 * np.sin(th1) - R2 * np.sin(th2) + yc
        return [eq1, eq2]

    # 提供一个初始猜测值 (radians) 来寻找一个解
    # 根据 P_c 在第二象限，猜测 R1 指向右上方，R2 指向左上方
    initial_guess = np.deg2rad([45, 135]) 
    try:
        th1, th2 = fsolve(position_equations, initial_guess)
    except Exception as e:
        print(f"位置分析求解失败: {e}")
        return None

    s1, c1 = np.sin(th1), np.cos(th1)
    s2, c2 = np.sin(th2), np.cos(th2)

    # --- 2. 速度分析 (求解 omega_1, omega_2) ---
    # 建立线性方程组: A_mat * w = b_vel
    # [-R1*s1, R2*s2] [w1] = [-v_cx]
    # [ R1*c1,-R2*c2] [w2] = [-v_cy]
    A_mat = np.array([
        [-R1 * s1, R2 * s2],
        [ R1 * c1, -R2 * c2]
    ])
    b_vel = np.array([-v_cx, -v_cy])
    
    try:
        w1, w2 = np.linalg.solve(A_mat, b_vel)
    except np.linalg.LinAlgError:
        print("错误: 速度分析矩阵是奇异的，无法求解。机构可能处于死点位置。")
        return None

    # --- 3. 加速度分析 (求解 alpha_1, alpha_2) ---
    # 建立线性方程组: A_mat * alpha = b_acc
    # A_mat 矩阵与速度分析相同
    # b_acc_x = -a_cx + w1^2*R1*c1 - w2^2*R2*c2
    # b_acc_y = -a_cy + w1^2*R1*s1 - w2^2*R2*s2
    b_acc = np.array([
        -a_cx + w1**2 * R1 * c1 - w2**2 * R2 * c2,
        -a_cy + w1**2 * R1 * s1 - w2**2 * R2 * s2
    ])
    
    try:
        alpha1, alpha2 = np.linalg.solve(A_mat, b_acc)
    except np.linalg.LinAlgError:
        print("错误: 加速度分析矩阵是奇异的，无法求解。机构可能处于死点位置。")
        return None

    results = {
        'theta_1_deg': np.rad2deg(th1),
        'theta_2_deg': np.rad2deg(th2),
        'omega_1_rad_s': w1,
        'omega_2_rad_s': w2,
        'alpha_1_rad_s2': alpha1,
        'alpha_2_rad_s2': alpha2
    }
    
    return results

# --- 主程序 ---
if __name__ == "__main__":
    # 连杆长度参数
    R1 = 3.0
    R2 = 2.0
    
    # *** 水球的完整运动状态 (来自您的图片) ***
    P_c_input = (-1.389, 2.079)
    V_c_input = (9.028, 6.032)
    A_c_input = (16.898, -45.420)

    print("--- 输入参数 ---")
    print(f"R1 = {R1}, R2 = {R2}")
    print(f"水球位置 P_c: {P_c_input} m")
    print(f"水球速度 V_c: {V_c_input} m/s")
    print(f"水球加速度 A_c: {A_c_input} m/s^2\n")

    # 求解运动学
    solution = solve_general_kinematics(R1, R2, P_c_input, V_c_input, A_c_input)

    if solution:
        print("--- 求解结果 ---")
        # 对角度进行标准化，使其在 (-180, 180] 范围内
        theta_1_norm = (solution['theta_1_deg'] + 180) % 360 - 180
        theta_2_norm = (solution['theta_2_deg'] + 180) % 360 - 180

        print(f"角度 theta_1: {theta_1_norm:.2f} 度")
        print(f"角度 theta_2: {theta_2_norm:.2f} 度")
        print("-" * 20)
        print(f"角速度 omega_1: {solution['omega_1_rad_s']:.4f} rad/s")
        print(f"角速度 omega_2: {solution['omega_2_rad_s']:.4f} rad/s")
        print("-" * 20)
        print(f"角加速度 alpha_1: {solution['alpha_1_rad_s2']:.4f} rad/s^2")
        print(f"角加速度 alpha_2: {solution['alpha_2_rad_s2']:.4f} rad/s^2")
        R3 = 2.5
        G = 0 + 1.5j # 假设基点 G 是坐标原点

        # 2. 从 solution 字典中获取所需的运动学变量
        theta_2_deg = theta_2_norm
        omega_2 = solution['omega_1_rad_s']
        alpha_2 = solution['alpha_1_rad_s2']

        # 3. 将角度从度转换为弧度
        theta_2_rad = np.deg2rad(theta_2_deg)

        # 4. 使用复数进行计算
        # 创建表示方向的单位复数 e^(j*theta)
        unit_vector_complex = np.cos(theta_2_rad) + 1j * np.sin(theta_2_rad)

        # 计算位置矢量 r
        r_complex = G + R3 * unit_vector_complex

        # 计算速度矢量 v = j*w*r
        v_complex = 1j * omega_2 * (R3 * unit_vector_complex)

        # 计算加速度矢量 a = j*alpha*r - w^2*r
        a_complex = (1j * alpha_2 * (R3 * unit_vector_complex)) - (omega_2**2 * (R3 * unit_vector_complex))

        # 5. 打印结果
        print(f"位置 r (x, y):    ({r_complex.real:.4f}, {r_complex.imag:.4f}) m")
        print(f"速度 v (vx, vy):  ({v_complex.real:.4f}, {v_complex.imag:.4f}) m/s")
        print(f"加速度 a (ax, ay): ({a_complex.real:.4f}, {a_complex.imag:.4f}) m/s^2")



--- 输入参数 ---
R1 = 3.0, R2 = 2.0
水球位置 P_c: (-1.389, 2.079) m
水球速度 V_c: (9.028, 6.032) m/s
水球加速度 A_c: (16.898, -45.42) m/s^2

--- 求解结果 ---
角度 theta_1: -1.57 度
角度 theta_2: 86.85 度
--------------------
角速度 omega_1: -2.1738 rad/s
角速度 omega_2: -4.4315 rad/s
--------------------
角加速度 alpha_1: 1.8465 rad/s^2
角加速度 alpha_2: -2.5221 rad/s^2
位置 r (x, y):    (0.1373, 3.9962) m
速度 v (vx, vy):  (5.4263, -0.2986) m/s
加速度 a (ax, ay): (-5.2583, -11.5420) m/s^2


In [35]:
from math import *
3/2/(1-log(2))
# 6*(1-log(2))

4.888337029906394

In [ ]:
3/

In [19]:
import numpy as np
from scipy.optimize import fsolve

def solve_primary_dyad(R1, R2, P_c, V_c, A_c):
    """
    求解由点C驱动的第一个dyad (C-J-G) 的运动学。
    返回铰链点J的位置、速度、加速度。
    """
    xc, yc = P_c
    v_cx, v_cy = V_c
    a_cx, a_cy = A_c

    # --- 位置分析 for theta1, theta2 ---
    def pos_eqs(vars):
        th1, th2 = vars
        return [R1 * np.cos(th1) - R2 * np.cos(th2) + xc,
                R1 * np.sin(th1) - R2 * np.sin(th2) + yc]

    initial_guess = np.deg2rad([45, 135])
    try:
        th1, th2 = fsolve(pos_eqs, initial_guess)
    except Exception: return None
    s1, c1 = np.sin(th1), np.cos(th1)
    s2, c2 = np.sin(th2), np.cos(th2)
    
    # --- 速度分析 for omega1, omega2 ---
    A_mat = np.array([[-R1*s1, R2*s2], [R1*c1, -R2*c2]])
    b_vel = np.array([-v_cx, -v_cy])
    try:
        w1, w2 = np.linalg.solve(A_mat, b_vel)
    except np.linalg.LinAlgError: return None

    # --- 加速度分析 for alpha1, alpha2 ---
    b_acc = np.array([-a_cx + w1**2*R1*c1 - w2**2*R2*c2,
                      -a_cy + w1**2*R1*s1 - w2**2*R2*s2])
    try:
        alpha1, alpha2 = np.linalg.solve(A_mat, b_acc)
    except np.linalg.LinAlgError: return None

    # --- 计算铰链点 J 的运动状态 ---
    P_j = np.array([R2 * c2, R2 * s2])
    V_j = np.array([-R2 * w2 * s2, R2 * w2 * c2])
    A_j_tan = np.array([-R2 * alpha2 * s2, R2 * alpha2 * c2])
    A_j_norm = np.array([-R2 * w2**2 * c2, -R2 * w2**2 * s2])
    A_j = A_j_tan + A_j_norm
    
    return {'P_j': P_j, 'V_j': V_j, 'A_j': A_j, 'th2': th2, 'w2': w2, 'alpha2': alpha2}


def solve_secondary_dyad(L_JF, L_EF, P_E, primary_kinematics):
    """
    求解由点J驱动的第二个dyad (J-F-E) 的运动学。
    返回点F的位置、速度、加速度。
    """
    P_j, V_j, A_j = primary_kinematics['P_j'], primary_kinematics['V_j'], primary_kinematics['A_j']
    P_ex, P_ey = P_E
    
    # --- 位置分析 for theta_JF, theta_EF ---
    def pos_eqs_secondary(vars):
        th_jf, th_ef = vars
        eq1 = P_j[0] + L_JF * np.cos(th_jf) - P_ex - L_EF * np.cos(th_ef)
        eq2 = P_j[1] + L_JF * np.sin(th_jf) - P_ey - L_EF * np.sin(th_ef)
        return [eq1, eq2]
        
    initial_guess = np.deg2rad([30, 60])
    try:
        th_jf, th_ef = fsolve(pos_eqs_secondary, initial_guess)
    except Exception: return None
    s_jf, c_jf = np.sin(th_jf), np.cos(th_jf)
    s_ef, c_ef = np.sin(th_ef), np.cos(th_ef)
    
    # --- 速度分析 for omega_JF, omega_EF ---
    A_mat = np.array([[-L_JF*s_jf, L_EF*s_ef], [L_JF*c_jf, -L_EF*c_ef]])
    b_vel = np.array([V_j[0], V_j[1]])
    try:
        w_jf, w_ef = np.linalg.solve(A_mat, b_vel)
    except np.linalg.LinAlgError: return None

    # --- 加速度分析 for alpha_JF, alpha_EF ---
    b_acc = np.array([A_j[0] + L_JF * w_jf**2 * c_jf - L_EF * w_ef**2 * c_ef,
                      A_j[1] + L_JF * w_jf**2 * s_jf - L_EF * w_ef**2 * s_ef])
    try:
        alpha_jf, alpha_ef = np.linalg.solve(A_mat, b_acc)
    except np.linalg.LinAlgError: return None
    
    # --- 计算点 F 的运动状态 ---
    P_f = np.array([P_ex + L_EF * c_ef, P_ey + L_EF * s_ef])
    V_f = np.array([-L_EF * w_ef * s_ef, L_EF * w_ef * c_ef])
    A_f_tan = np.array([-L_EF * alpha_ef * s_ef, L_EF * alpha_ef * c_ef])
    A_f_norm = np.array([-L_EF * w_ef**2 * c_ef, -L_EF * w_ef**2 * s_ef])
    A_f = A_f_tan + A_f_norm
    
    return {'P_f': P_f, 'V_f': V_f, 'A_f': A_f}

# --- 主程序 ---
if __name__ == "__main__":
    # --- 1. 定义已知参数 ---
    # 主机构 (C-J-G)
    R1 = 3.0
    R2 = 2.0
    P_c_input = (-1.389, 2.079)
    V_c_input = (9.028, 6.032)
    A_c_input = (16.898, -45.420)
    
    # 次级机构 (J-F-E) - 包含假设
    L_JF = 3.0  # (假设) 连杆 JF 的长度
    L_EF = 2.5  # (给定) 连杆 EF 的长度
    P_E_input = (1.5, 0.0) # (假设) 固定点 E 的位置

    print("--- 假设 ---")
    print(f"新增固定铰链点 E 的位置: {P_E_input} m")
    print(f"新增连杆 JF 的长度: {L_JF} m\n")

    # --- 2. 求解第一部分 ---
    primary_kinematics = solve_primary_dyad(R1, R2, P_c_input, V_c_input, A_c_input)
    
    if primary_kinematics:
        print("--- 第一级机构求解成功 (铰链点J的运动) ---")
        print(f"位置 P_j: ({primary_kinematics['P_j'][0]:.3f}, {primary_kinematics['P_j'][1]:.3f}) m")
        print(f"速度 V_j: ({primary_kinematics['V_j'][0]:.3f}, {primary_kinematics['V_j'][1]:.3f}) m/s")
        print(f"加速度 A_j: ({primary_kinematics['A_j'][0]:.3f}, {primary_kinematics['A_j'][1]:.3f}) m/s^2\n")

        # --- 3. 求解第二部分 ---
        final_kinematics = solve_secondary_dyad(L_JF, L_EF, P_E_input, primary_kinematics)
        
        if final_kinematics:
            print("--- 最终结果 (点F的运动状态) ---")
            P_f = final_kinematics['P_f']
            V_f = final_kinematics['V_f']
            A_f = final_kinematics['A_f']
            
            print(f"位置 r (x, y):    ({P_f[0]:.4f}, {P_f[1]:.4f}) m")
            print(f"速度 v (vx, vy):  ({V_f[0]:.4f}, {V_f[1]:.4f}) m/s")
            print(f"速率 |v|:         {np.linalg.norm(V_f):.4f} m/s\n")
            print(f"加速度 a (ax, ay): ({A_f[0]:.4f}, {A_f[1]:.4f}) m/s^2")
            print(f"加速度大小 |a|:   {np.linalg.norm(A_f):.4f} m/s^2")
        else:
            print("求解失败：无法计算次级机构 (J-F-E) 的运动。")
    else:
        print("求解失败：无法计算主机构 (C-J-G) 的运动。")

--- 假设 ---
新增固定铰链点 E 的位置: (1.5, 0.0) m
新增连杆 JF 的长度: 3.0 m

--- 第一级机构求解成功 (铰链点J的运动) ---
位置 P_j: (1.511, 1.310) m
速度 V_j: (5.690, -6.561) m/s
加速度 A_j: (-34.349, -17.955) m/s^2

--- 最终结果 (点F的运动状态) ---
位置 r (x, y):    (3.9654, -0.4145) m
速度 v (vx, vy):  (3.2392, 19.2677) m/s
速率 |v|:         19.5381 m/s

加速度 a (ax, ay): (-179.4597, -146.4568) m/s^2
加速度大小 |a|:   231.6364 m/s^2
